In [2]:
import os

import pandas as pd
import haven.db as db 

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [5]:
sql = '''
with neighbors as (
    select 
        *,
        row_number() over () as _choice
    from 
        movement_model_neighbors
    where
        max_km = 100
)
select 
    n.current_h3_index,
    n.selected_h3_index,
    n.neighbor_h3_index,
    n.date,
    n._individual,
    n._decision,
    n._choice,
    n.tag_key,
    p.mixed_layer_thickness,
    p.salinity,
    p.temperature,
    p.velocity_east,
    p.velocity_north
from 
    neighbors n 
    inner join copernicus_physics p 
        on p.depth_bin = 25 
        and region = 'chinook_study'
        and n.neighbor_h3_index = p.h3_index 
        and n.date = p.date 
'''

cached_file_path = 'cached.snappy.parquet'
if os.path.exists(cached_file_path):
    data = pd.read_parquet(cached_file_path)
else:
    data = db.read_data(sql)
data.to_parquet(cached_file_path)

AttributeError: 'DataFrame' object has no attribute 'write_parquet'